In [1]:
import pandas as pd
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
dataset=pd.read_csv("balanced_dataset.csv")

In [3]:
dataset

,app_name,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,NIP,18,22,28,38,48,8000,1
1,NIP,14,41,24,34,44,4000,0
2,NIP,17,38,27,37,47,7000,0
3,NIP,15,97,25,35,45,5000,0
4,NIP,11,67,21,31,41,1000,0
...,...,...,...,...,...,...,...,...
1815,NIP,22,48,32,42,52,12000,1
1816,NIP,18,82,28,38,48,8000,1
1817,NIP,17,97,27,37,47,7000,0
1818,NIP,18,53,28,38,48,8000,1


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,18,22,28,38,48,8000,1
1,14,41,24,34,44,4000,0
2,17,38,27,37,47,7000,0
3,15,97,25,35,45,5000,0
4,11,67,21,31,41,1000,0
...,...,...,...,...,...,...,...
1815,22,48,32,42,52,12000,1
1816,18,82,28,38,48,8000,1
1817,17,97,27,37,47,7000,0
1818,18,53,28,38,48,8000,1


In [6]:
dataset.columns

Index(['response_time', 'error_rate', 'cpu_usage', 'memory_usage',
       'disk_space', 'active_users', 'downtime'],
      dtype='object')

In [7]:
indep=dataset[['response_time', 'error_rate', 'cpu_usage', 'memory_usage',
       'disk_space', 'active_users']]

In [8]:
dep=dataset[['downtime']]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split( indep, dep, test_size=0.30, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(x_train)
X_test=sc.transform(x_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy','log_loss'],'max_features': ['None','sqrt','log2'],'class_weight':['balanced','balanced_subsample','None']}
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample',
                                          'None'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['None', 'sqrt', 'log2']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("the f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

the f1_macro value for best parameter{'class_weight': 'balanced', 'criterion': 'entropy', 'max_features': 'log2'}: 0.8804922326811344


In [15]:
print("the confusion matrix:\n",cm)

the confusion matrix:
 [[222  48]
 [ 17 259]]


In [16]:
print("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.93      0.82      0.87       270
           1       0.84      0.94      0.89       276

    accuracy                           0.88       546
   macro avg       0.89      0.88      0.88       546
weighted avg       0.89      0.88      0.88       546



In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9102053140096618